<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## make life easier with defaults  

In [ ]:
defaults = type('defaults', (object,), {'margin': 157, # align to the right by 157
                                        'orisrc': None, # keep a copy of original official src code
                                        'outenv': globals(), # outside global env
                                        'cmts': {} # a dict to store idx and cmt
                                        # 'eg': None, # examples
                                        # 'src': None, # official src
                                       })

## globals() and locals()

Interesting behavior of [locals()](https://stackoverflow.com/questions/7969949/whats-the-difference-between-globals-locals-and-vars)

In [ ]:
locals() == globals()

True

In [ ]:
"functools" in locals()

False

In [ ]:
"inspect" in locals()

False

In [ ]:
"eval" in locals()

False

In [ ]:
"defaults" in locals()

True

In [ ]:
"whatinside" in locals()

False

In [ ]:
"whichversion" in locals()

False

In [ ]:
from fastdebug.utils import *

In [ ]:
"whatinside" in locals()

True

In [ ]:
"whichversion" in globals()

True

## Execute strings

In [ ]:
eval?

Signature: eval(source, globals=None, locals=None, /)
Docstring:
Evaluate the given source in the context of globals and locals.

The source may be a string representing a Python expression
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method


In [ ]:
exec?

Signature: exec(source, globals=None, locals=None, /)
Docstring:
Execute the given source in the context of globals and locals.

The source may be a string representing one or more Python statements
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method


### new variable or updated variable by exec will only be accessible from locals()

In [ ]:
x = 1
def test():
    a = "1+x"
    pprint(f'locals: {locals()}', width=157) # x is not in locals() but in globals()
    print(eval(a)) 
test()

"locals: {'a': '1+x'}"
2


In [ ]:
x = 1
def test():
    a = "b = 1+x"
    pprint(locals())
    print(f'exec(a): {exec(a)}') 
    pprint(locals())
    print(b) # b can't be accessed directly and not use the key b

try:
    test()
except NameError as e:
    print(e)

{'a': 'b = 1+x'}
exec(a): None
{'a': 'b = 1+x', 'b': 2}
name 'b' is not defined


In [ ]:
x = 1
def test():
    a = "b = 1+x"
    pprint(locals())
    print(f'exec(a): {exec(a)}') 
    pprint(locals())
    b = locals()['b'] # you can't even assign the value to b, otherwise b:value won't be available
    print(b)

# test()
try:
    test()
except KeyError as e:
    print("KeyError: 'b' does not exist")

{'a': 'b = 1+x'}
exec(a): None
{'a': 'b = 1+x'}
KeyError: 'b' does not exist


In [ ]:
x = 1
def test():
    a = "b = 1+x"
    pprint(locals())
    print(f'exec(a): {exec(a)}') 
    pprint(locals())
    c = locals()['b'] # if assign to a different name, then b is available
    print(c)
    pprint(locals())

# test()
try:
    test()
except KeyError as e:
    print("KeyError: 'b' does not exist")

{'a': 'b = 1+x'}
exec(a): None
{'a': 'b = 1+x', 'b': 2}
2
{'a': 'b = 1+x', 'b': 2, 'c': 2}


In [ ]:
x = 1
def test():
    a = "b = 1+x"
    pprint(locals())
    print(f'exec(a): {exec(a)}') 
    pprint(locals())
    c = locals()['b'] # if assign to a different name, then b is available
    print(f'c = locals()["b"]; c: {c}')
    pprint(locals())
    print(f'exec(c = c + 1): {exec("c = c + 1")}') # update c won't change anything
    pprint(locals())

test()

{'a': 'b = 1+x'}
exec(a): None
{'a': 'b = 1+x', 'b': 2}
c = locals()["b"]; c: 2
{'a': 'b = 1+x', 'b': 2, 'c': 2}
exec(c = c + 1): None
{'a': 'b = 1+x', 'b': 2, 'c': 2}


In [ ]:
x = 1
def test():
    a = "b = 1+x"
    pprint(locals())
    print(f'exec(a): {exec(a)}') 
    pprint(locals())
    c = locals()['b'] # if assign to a different name, then b is available
    print(f'c = locals()["b"]; c: {c}')
    pprint(locals())
    print(f'exec(d = c + 1): {exec("d = c + 1")}') # must assign to a different variable, not c anymore
    pprint(locals())

test()

{'a': 'b = 1+x'}
exec(a): None
{'a': 'b = 1+x', 'b': 2}
c = locals()["b"]; c: 2
{'a': 'b = 1+x', 'b': 2, 'c': 2}
exec(d = c + 1): None
{'a': 'b = 1+x', 'b': 2, 'c': 2, 'd': 3}


### eval can override its own globals() and locals()

In [ ]:
def test():
    a = 1
    b = "a+1"
    pprint(f'locals: {locals()}', width=157)
    print(f'b: {eval(b, {}, {"a": 2})}') # globals() put to empty and locals() to include 'a' with a different value
    pprint(f'locals: {locals()}', width=157) 

test()

"locals: {'a': 1, 'b': 'a+1'}"
b: 3
"locals: {'a': 1, 'b': 'a+1'}"


### when exec update existing functions

In [ ]:
def foo(x, y): return x + y
def test(func):
    a = 1
    b = inspect.getsource(func)
    newb = ""
    for l in b.split('\n'):
        if bool(l):
            newb = newb + l
    newb = newb + " + 3\n"
    pprint(f'locals: {locals()}', width=157) # foo is not available in locals()
    print(f'exec(newb): {exec(newb)}') 
    pprint(f'locals: {locals()}', width=157) # a foo is available in locals(), but which one is it
    newfoo = locals()["foo"]
    print(newfoo(1,9))
    print(locals()["foo"](1,9))
    print(func(1,9))
    print(foo(1,9))

test(foo)

"locals: {'func': <function foo at 0x1182fd670>, 'a': 1, 'b': 'def foo(x, y): return x + y\\n', 'newb': 'def foo(x, y): return x + y + 3\\n', 'l': ''}"
exec(newb): None
("locals: {'func': <function foo at 0x1182fd670>, 'a': 1, 'b': 'def foo(x, y): return x + y\\n', 'newb': 'def foo(x, y): return x + y + 3\\n', 'l': '', "
 "'foo': <function foo at 0x1182fd310>}")
13
13
10
10


### when the func to be udpated involve other libraries

In [ ]:
import functools

In [ ]:
def foo(x, y): 
    print(inspect.signature(foo))
    return x + y
def test(func):
    a = 1
    b = inspect.getsource(func)
    newb = ""
    for l in b.split('\n'):
        if bool(l) and "return" not in l :
            newb = newb + l + '\n'
        elif bool(l):
            newb = newb + l
    newb = newb + " + 3\n"
    pprint(f'locals: {locals()}', width=157) # foo is not available in locals()
    print(f'exec(newb): {exec(newb)}') 
    pprint(f'locals: {locals()}', width=157) # a foo is available in locals(), but which one is it
    newfoo = locals()["foo"]
    print(newfoo(1,9))
    print(locals()["foo"](1,9))
    print(func(1,9))
    print(foo(1,9))

test(foo)

("locals: {'func': <function foo at 0x1182fdd30>, 'a': 1, 'b': 'def foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y\\n', 'newb': 'def "
 "foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y + 3\\n', 'l': ''}")
exec(newb): None
("locals: {'func': <function foo at 0x1182fdd30>, 'a': 1, 'b': 'def foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y\\n', 'newb': 'def "
 "foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y + 3\\n', 'l': '', 'foo': <function foo at 0x1182fd940>}")
(x, y)
13
(x, y)
13
(x, y)
10
(x, y)
10


### inside a function, exec() allow won't give you necessary env from function namespace

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)
    exec(b) # create the new add in locals
    pprint(f'locals: {locals()}', width=157)
    print(f'add(5,6): {add(5,6)}')
    add1 = locals()['add'] # assign a different name, add1
    
    print(f'add1(5,6): {add1(5,6)}') # error: lst is not defined, even though lst=[] is right above
    
    pprint(f'locals: {locals()}', width=157)

try:
    test(add)
except NameError as e:
    print(e)

"locals: {'func': <function add at 0x1182fdf70>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
("locals: {'func': <function add at 0x1182fdf70>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x1183000d0>}')
add(5,6): 1
name 'lst' is not defined


### magic of `exec(b, globals().update(locals()))`

What about `exec(b, globals().update(globals()))`

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)
    
    exec(b, globals().update(globals())) # update(globals()) won't give us lst above
    
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] 
    print(add1(5,6))
    pprint(f'locals: {locals()}', width=157)



try:
    test(add)
except: 
    print("exec(b, globals().update(globals())) won't give us lst in the func namespace")

"locals: {'func': <function add at 0x1183003a0>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
("locals: {'func': <function add at 0x1183003a0>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x118300310>}')
exec(b, globals().update(globals())) won't give us lst in the func namespace


In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)
    
    exec(b, globals().update(locals())) # make sure b can access lst from above
    
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] 
    print(add1(5,6))
    pprint(f'locals: {locals()}', width=157)

test(add)
print(add(5,6))
try:
    print(add1(5,6))
except: 
    print("you can't bring add1 from a function namespace to the outside world")

"locals: {'func': <function add at 0x118300700>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
("locals: {'func': <function add at 0x118300700>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x1183008b0>}')
11
("locals: {'func': <function add at 0x118300700>, 'a': 1, 'lst': [5], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 "0x1183008b0>, 'add1': <function add at 0x1183008b0>}")
1
you can't bring add1 from a function namespace to the outside world


### Bring variables from a func namespace to the sideout world

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)    
    
    exec(b, globals().update(locals())) # make sure b can access lst from above

    add1 = locals()['add'] 
    print(add1(5,6))
    add1(5,6)
    pprint(f'locals: {locals()}', width=157)

    # bring variables inside a func to the outside
    globals().update(locals())

test(add)
pprint(add(5,6)) # the original add is override by the add from the function's locals()
pprint(add1(5,6))
print(lst)

"locals: {'func': <function add at 0x118300af0>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
11
("locals: {'func': <function add at 0x118300af0>, 'a': 1, 'lst': [5, 5], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add "
 "at 0x118300700>, 'add1': <function add at 0x118300700>}")
11
11
[5, 5, 5, 5]


### globals() in a cell vs globals() in a func

In [ ]:
from fastdebug.utils import tstenv

In [ ]:
len(globals().keys())

87

In [ ]:
globals()['__name__']

'__main__'

In [ ]:
tstenv()

out global env has 20 vars
inner global env has 20 vars
inner local env has 20 vars
['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', '__all__', 'os']
['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', '__all__', 'os']
out env['__name__']: fastdebug.utils
inner env['__name__']: fastdebug.utils


## make a colorful string

In [1]:
#|output: asis
#| echo: false
show_doc(dbcolors)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L26){target="_blank" style="float:right; font-size:smaller"}

### dbcolors

>      dbcolors ()

Initialize self.  See help(type(self)) for accurate signature.

In [2]:
#|output: asis
#| echo: false
show_doc(colorize)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L33){target="_blank" style="float:right; font-size:smaller"}

### colorize

>      colorize (cmt, color:str=None)

In [ ]:
colorize("this is me", "r")

'\x1bthis is me\x1b'

In [ ]:
print(colorize("this is me", "r"))

this is me


## align text to the most right

In [3]:
#|output: asis
#| echo: false
show_doc(strip_ansi)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L47){target="_blank" style="float:right; font-size:smaller"}

### strip_ansi

>      strip_ansi (source)

In [ ]:
def alignright(blocks):
    lst = blocks.split('\n')
    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))
    indent = defaults.margin - maxlen
    for l in lst:
        print(' '*indent + format(l))

In [4]:
#|output: asis
#| echo: false
show_doc(alignright)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L51){target="_blank" style="float:right; font-size:smaller"}

### alignright

>      alignright (blocks, margin:int=157)

In [ ]:
alignright("this is me")

                                                                                                                                                   this is me


## printsrcwithidx

### print the entire source code with idx from 0

In [ ]:
def printsrcwithidx(src):
    totallen = 157
    lenidx = 5
    lstsrc = inspect.getsource(src).split('\n')
    for idx, l in zip(range(len(lstsrc)), lstsrc):
        lenl = len(l)

        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")

### print the whole src with idx or print them in parts

In [ ]:
def printsrcwithidx(src, 
                    maxlines:int=33, # maximum num of lines per page
                    part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
    totallen = 157
    lenidx = 5
    lstsrc = inspect.getsource(src).split('\n')
    numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
    
    if part == 0: 
        for idx, l in zip(range(len(lstsrc)), lstsrc):
            lenl = len(l)
            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")

    for p in range(numparts):
        for idx, l in zip(range(len(lstsrc)), lstsrc):

            if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                lenl = len(l)
                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                
            if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                return

### use cmts from dbprint to print out src with comments

In [5]:
#|output: asis
#| echo: false
show_doc(printsrcwithidx)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L59){target="_blank" style="float:right; font-size:smaller"}

### printsrcwithidx

>      printsrcwithidx (src, maxlines:int=33, part:int=0)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| src |  |  |  |
| maxlines | int | 33 | maximum num of lines per page |
| part | int | 0 | if the src is more than 33 lines, then divide the src by 33 into a few parts |

In [ ]:
# printsrcwithidx(foo)

In [ ]:
# printsrcwithidx(pprint)

### no more update for printsrcwithidx, for the latest see Fastdb.print

see a [complex example](./examples/dbprint.ipynb#test-printsrcwithidx) on printsrcwithidx

## print out src code

### basic version

In [ ]:
def printsrc(src, srclines, cmt):
# print out the title
    print('\n')
    print('{:#^157}'.format(" srcline under investigation "))
    print('\n')


    # convert the source code of the function into a list of strings splitted by '\n'
    lst = inspect.getsource(src).split('\n')

    ccount = 0
    for l in lst:
        if bool(l) and l.strip() in srclines:# print out the srcline under investigation
            print('{:=<157}'.format(l))
            ccount = ccount + 1

            if bool(cmt): # print out comment at the end of the srclines under investigation
                numsrclines = len(srclines.split("\n"))
                if ccount == numsrclines:
                    colcmt = colorize(cmt, "r") # colorize the comment
                    alignright(colcmt) # put the comment to the most right

        else: 
            print('{:<157}'.format(l)) # print out the rest of source code

In [ ]:
def foo():     
    a = 1 + 1
    b = a + 1
    pass

In [ ]:
printsrc(foo, "a = 1 + 1", "this is comment")



################################################################ srcline under investigation ################################################################


def foo():                                                                                                                                                   
    a = 1 + 1================================================================================================================================================
                                                                                                                                              this is comment
    b = a + 1                                                                                                                                                
    pass                                                                                                                                                     
                                                

In [ ]:
printsrc(foo, "    a = 1 + 1\n    b = a + 1", "this is comment")



################################################################ srcline under investigation ################################################################


def foo():                                                                                                                                                   
    a = 1 + 1================================================================================================================================================
    b = a + 1================================================================================================================================================
                                                                                                                                              this is comment
    pass                                                                                                                                                     
                                                

### print src with specific number of lines

In [ ]:
len("    this is a code\nthis is another code".split('\n'))
list(map(lambda x: bool(x.strip()), "    this is a code\n    this is another code\n   \n   ".split('\n'))).count(True)

2

In [ ]:
def printsrc(src, srclines, cmt, expand:int=2):

    # convert the source code of the function into a list of strings splitted by '\n'
    lst = inspect.getsource(src).split('\n')
    
    # find out the idx of start srclines and end srclines
    startidx = 0
    numsrclines = list(map(lambda x: bool(x.strip()), srclines.split('\n'))).count(True)
    for idx, l in zip(range(len(lst)), lst):
        if bool(l) and l.strip() in srclines:
            startidx = idx
    endidx = startidx + numsrclines - 1
    
    ccount = 0
    for idx, l in zip(range(len(lst)), lst):
        if bool(l) and l.strip() in srclines:# print out the srcline under investigation with # as padding
            print('{:=<157}'.format(l))
            ccount = ccount + 1

            if bool(cmt): # print out comment at the end of the srclines under investigation
                # numsrclines = len(srclines.split("\n"))
                if ccount == numsrclines:
                    colcmt = colorize(cmt, "r") # colorize the comment
                    alignright(colcmt) # put the comment to the most right

        elif idx < startidx and idx >= startidx - expand:
            print('{:<157}'.format(l)) # print out the rest of source code to the most left

        elif idx <= endidx + expand and idx > endidx:
            print('{:<157}'.format(l)) # print out the rest of source code to the most left

### make the naming more sensible

In [ ]:
def printsrc(src, dblines, cmt, expand:int=2):

    lstsrc = inspect.getsource(src).split('\n')

    numdblines = list(map(lambda x: bool(x.strip()), dblines.split('\n'))).count(True)
    
    startdbidx = 0
    for idx, l in zip(range(len(lstsrc)), lstsrc):
        if bool(l) and l.strip() in dblines:
            startdbidx = idx
    enddbidx = startdbidx + numdblines - 1
    
    dbcount = 0
    for idx, l in zip(range(len(lstsrc)), lstsrc):
        if bool(l) and l.strip() in dblines:
            print('{:=<157}'.format(l))
            dbcount = dbcount + 1

            if bool(cmt): 
                if dbcount == numdblines:
                    colcmt = colorize(cmt, "r")
                    alignright(colcmt) # also print the comment

        elif idx < startdbidx and idx >= startdbidx - expand:
            print('{:<157}'.format(l)) 

        elif idx <= enddbidx + expand and idx > enddbidx:
            print('{:<157}'.format(l))

### Allow a dbline occur more than once

In [ ]:
def printsrc(src, dblines, cmt, expand:int=2):

    lstsrc = inspect.getsource(src).split('\n')

    numdblines = list(map(lambda x: bool(x.strip()), dblines.split('\n'))).count(True)
    
    dblineidx = []
    for idx, l in zip(range(len(lstsrc)), lstsrc):
        if bool(l) and l.strip() in dblines:
            # print(f'idx: {idx}; and l: {l}') # debugging
            dblineidx.append(idx)
    # enddbidx = startdbidx + numdblines - 1
    
    # dbcount = 0
    for idx, l in zip(range(len(lstsrc)), lstsrc):
        if bool(l) and l.strip() in dblines:
            # print(f'idx: {idx}; and l: {l}') # debugging
            print('{:=<157}'.format(l))
            # dbcount = dbcount + 1

            if bool(cmt): 
                # if dbcount == numdblines:
                    colcmt = colorize(cmt, "r")
                    alignright(colcmt) # also print the comment

        for o, dbidx in zip(range(len(dblineidx)), dblineidx):
            if idx >= dbidx - expand and idx < dbidx: 
                print('{:<157}'.format(l)) 
            elif idx <= dbidx + expand and idx > dbidx: 
                print('{:<157}'.format(l)) 
                
                if idx == dbidx + expand and len(dblineidx)>=2:
                    print("\n")
                    print('{:=^157}'.format(f" The occurance {o} "))
                    print("\n")

### adding idx to the selected srclines

#### printsrclinewithidx

In [6]:
#|output: asis
#| echo: false
show_doc(printsrclinewithidx)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L137){target="_blank" style="float:right; font-size:smaller"}

### printsrclinewithidx

>      printsrclinewithidx (idx, l, fill=' ')

In [ ]:
def printsrc(src, # name of src code such as foo, or delegates
             dblines,
             cmt,
             expand:int=2): # expand the codes around the srcline under investigation
    
    lstsrc = inspect.getsource(src).split('\n')
    numdblines = list(map(lambda x: bool(x.strip()), dblines.split('\n'))).count(True)
    
    dblineidx = []
    for idx, l in zip(range(len(lstsrc)), lstsrc):
        if bool(l) and l.strip() in dblines:
            dblineidx.append(idx)

    for idx, l in zip(range(len(lstsrc)), lstsrc):
        if bool(l) and l.strip() in dblines:
            printsrclinewithidx(idx, l, fill="=")

            if bool(cmt):
                colcmt = colorize(cmt, "r")
                alignright(colcmt) # also print the comment

        for o, dbidx in zip(range(len(dblineidx)), dblineidx):
            if idx >= dbidx - expand and idx < dbidx:
                printsrclinewithidx(idx, l)
            elif idx <= dbidx + expand and idx > dbidx:
                printsrclinewithidx(idx, l)
                
                if idx == dbidx + expand and len(dblineidx)>=2:
                    print("\n")
                    print('{:=^157}'.format(f" The occurance {o} "))
                    print("\n")

### dblines can be string of code or idx number

In [ ]:
type(int("120"))

int

In [ ]:
a = 120
type(a) == int

True

In [ ]:
def printsrc(src, # name of src code such as foo, or delegates
             dbcode, # string of codes or int of code idx number
             cmt,
             expand:int=2): # expand the codes around the srcline under investigation
    
    lstsrc = inspect.getsource(src).split('\n')
    
    dblines = ""
    if type(dbcode) == int:
        dblines = lstsrc[dbcode]
    else:
        dblines = dbcode
        numdblines = list(map(lambda x: bool(x.strip()), dblines.split('\n'))).count(True)
    
    dblineidx = []
    for idx, l in zip(range(len(lstsrc)), lstsrc):
        if bool(l) and l.strip() in dblines:
            dblineidx.append(idx)

    for idx, l in zip(range(len(lstsrc)), lstsrc):
        if bool(l) and l.strip() in dblines:
            printsrclinewithidx(idx, l, fill="=")

            if bool(cmt):
                colcmt = colorize(cmt, "r")
                alignright(colcmt) # also print the comment

        for o, dbidx in zip(range(len(dblineidx)), dblineidx):
            if idx >= dbidx - expand and idx < dbidx:
                printsrclinewithidx(idx, l)
            elif idx <= dbidx + expand and idx > dbidx:
                printsrclinewithidx(idx, l)
                
                if idx == dbidx + expand and len(dblineidx)>=2:
                    print("\n")
                    print('{:=^157}'.format(f" The occurance {o} "))
                    print("\n")

### avoid multi-occurrance of the same srcline

In [7]:
#|output: asis
#| echo: false
show_doc(printsrc)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L144){target="_blank" style="float:right; font-size:smaller"}

### printsrc

>      printsrc (src, dbcode, cmt, expand:int=2)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| src |  |  | name of src code such as foo, or delegates |
| dbcode |  |  | string of codes or int of code idx number |
| cmt |  |  |  |
| expand | int | 2 | expand the codes around the srcline under investigation |

In [ ]:
def foo(a):
    "this is docs"
    # this is a pure comment
    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        "this is docs"
        b = a + 1
    "this is docs"
    return a

In [ ]:
printsrc(foo, "else:", "this is comment", expand=4)

    # this is a pure comment                                                                                                                            (2)
    if a > 1:                                                                                                                                           (3)
        for i in range(3):                                                                                                                              (4)
            a = i + 1                                                                                                                                   (5)
    else:===============================================================================================================================================(6)
                                                                                                                                              this is comment
        "this is docs"                                        

In [ ]:
printsrc(foo, "return a", "this is comment", expand=10)

def foo(a):                                                                                                                                             (0)
    "this is docs"                                                                                                                                      (1)
    # this is a pure comment                                                                                                                            (2)
    if a > 1:                                                                                                                                           (3)
        for i in range(3):                                                                                                                              (4)
            a = i + 1                                                                                                                                   (5)
    else:                                                       

In [ ]:
printsrc(foo, 3, "this is comment")

    "this is docs"                                                                                                                                      (1)
    # this is a pure comment                                                                                                                            (2)
    if a > 1:===========================================================================================================================================(3)
                                                                                                                                              this is comment
        for i in range(3):                                                                                                                              (4)
            a = i + 1                                                                                                                                   (5)


more [complex example](./examples/printsrc.ipynb) on printsrc

## dbprint on expression

### basic version

In [ ]:
def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "debug a srcline with one or more expressions with src printed."
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt)
    
    for c in code:
    # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
        output = f"{c} => {c} : {eval(c, globals().update(env))}"
        print('{:>157}'.format(output))

In [ ]:
def foo():     
    a = 1 + 1
    b = a + 1
    pass

In [ ]:
def foo(): 
    dbprint(foo, "    a = 1 + 1", "this is a test", "1+2", "str(1+2)")
    a = 1 + 1
    pass

In [ ]:
foo()

def foo():                                                                                                                                              (0)
    dbprint(foo, "    a = 1 + 1", "this is a test", "1+2", "str(1+2)")                                                                                  (1)
    a = 1 + 1===========================================================================================================================================(2)
                                                                                                                                               this is a test
    pass                                                                                                                                                (3)
                                                                                                                                                        (4)
                                                              

### insert dbcode and make a new dbfunc

when bringing back splitted lines back, we need add '\n' back to them

In [ ]:
back = ""
for l in inspect.getsource(foo).split('\n'):
    back = back + l
pprint(back, width=157)

'def foo():     dbprint(foo, "    a = 1 + 1", "this is a test", "1+2", "str(1+2)")    a = 1 + 1    pass'


In [ ]:
def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2 # span 2 lines of srcode up and down from the srcline investigated
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ")"
                elif count == len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ")"
                elif count != len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ","
                elif count != len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
                

    # print out the new srcode
    # for l in dbsrc.split('\n'):
    #     print(l)
    
    # exec the dbsrc to replace the official source code
    exec(dbsrc) # created new foo and saved inside locals()

    
    # check to see whether the new srcode is created
    # print(locals())
    
    # move this new foo into globals, so that outside the cell we can still use it
    globals().update(locals())
    
    return locals()[defaults.orisrc.__name__]

### Bring outside namespace variables into exec()

In [ ]:
def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            env = globals() # outer env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ")"
                elif count == len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ")"
                elif count != len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ","
                elif count != len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
                

    # print out the new srcode
    # for l in dbsrc.split('\n'):
    #     print(l)
    
    # exec the dbsrc to replace the official source code
    # exec(dbsrc) # created new foo and saved inside locals()
    # exec(dbsrc, globals().update(locals())) # make sure b can access lst from above
    exec(dbsrc, globals().update(env)) # make sure b can access lst from above
    
    # check to see whether the new srcode is created
    # print(f'locals()["src"]: {locals()["src"]}')
    # print(f'locals()["{src.__name__}"]: {locals()[src.__name__]}')
    
    # move this new foo into globals, so that outside the cell we can still use it
    globals().update(locals())
    
    return locals()[defaults.orisrc.__name__]

### Bring what inside the func namespace variables to the outside world

In [ ]:
def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            env = globals() # outer env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ")"
                elif count == len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ")"
                elif count != len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ","
                elif count != len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
                

    exec(dbsrc, globals().update(env)) # make sure b can access lst from above
    
    # check to see whether the new srcode is created
    # print(f'locals()["src"]: {locals()["src"]}')
    # print(f'locals()["{src.__name__}"]: {locals()[src.__name__]}')
    
    # this is crucial to bring what inside a func namespace into the outside world
    env.update(locals())
    
    return locals()[defaults.orisrc.__name__]

see a [complex example](./examples/dbprint.ipynb) on dbprint

### Adding g = locals() to dbprintinsert to avoid adding env individually

In [ ]:
def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            env = {} # outer env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ")"
                elif count == len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ")"
                elif count != len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ","
                elif count != len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n' # adding this line above dbprint line
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
    
    # print out the new srcode
    # for l in dbsrc.split('\n'):
    #     print(l)

    exec(dbsrc, globals().update(env)) # make sure b can access lst from above
    
    # check to see whether the new srcode is created
    # print(f'locals()["src"]: {locals()["src"]}')
    # print(f'locals()["{src.__name__}"]: {locals()[src.__name__]}')
    
    # this is crucial to bring what inside a func namespace into the outside world
    env.update(locals())
    
    return locals()[defaults.orisrc.__name__]

### enable srclines to be either string or int 

In [ ]:
def dbprint(src, # the src func name, e.g., foo
            dbcode, # the srclines under investigation, can be either string or int
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            env = {} # outer env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, dbcode, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # dbprint is enabled to accept both string or int
    srclines = ""
    if type(dbcode) == int:
        srclines = lst[dbcode]
    else:
        srclines = dbcode
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ")"
                elif count == len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ")"
                elif count != len(codes) and "=" in c:
                    dbcodes = dbcodes + c + ","
                elif count != len(codes) and "=" not in c:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n' # adding this line above dbprint line
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
    
    # print out the new srcode
    # for l in dbsrc.split('\n'):
    #     print(l)

    exec(dbsrc, globals().update(env)) # make sure b can access lst from above
    
    # check to see whether the new srcode is created
    # print(f'locals()["src"]: {locals()["src"]}')
    # print(f'locals()["{src.__name__}"]: {locals()[src.__name__]}')
    
    # this is crucial to bring what inside a func namespace into the outside world
    env.update(locals())
    
    return locals()[defaults.orisrc.__name__]

### enable = to be used as assignment in codes

In [ ]:
def dbprint(src, # the src func name, e.g., foo
            dbcode, # the srclines under investigation, can be either string or int
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            env = {} # outer env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, dbcode, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # dbprint is enabled to accept both string or int
    srclines = ""
    if type(dbcode) == int:
        srclines = lst[dbcode]
    else:
        srclines = dbcode
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes) and "env=" in c:
                    dbcodes = dbcodes + c + ")"
                # elif count == len(codes) and "=" not in c:
                #     dbcodes = dbcodes + '"' + c + '"' + ")"
                # elif count != len(codes) and "=" in c:
                #     dbcodes = dbcodes + c + ","
                # elif count != len(codes) and "=" not in c:
                else:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n' # adding this line above dbprint line
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
    
    # print out the new srcode
    # for l in dbsrc.split('\n'):
    #     print(l)

    exec(dbsrc, globals().update(env)) # make sure b can access lst from above
    
    # check to see whether the new srcode is created
    # print(f'locals()["src"]: {locals()["src"]}')
    # print(f'locals()["{src.__name__}"]: {locals()[src.__name__]}')
    
    # this is crucial to bring what inside a func namespace into the outside world
    env.update(locals())
    
    return locals()[defaults.orisrc.__name__]

### avoid adding "env=g" for dbprintinsert

In [ ]:
def dbprint(src, # the src func name, e.g., foo
            dbcode, # the srclines under investigation, can be either string or int
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            env = {} # outer env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"
    
    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
    
    
    # print out src code: the basic version
    printsrc(src, dbcode, cmt, expand)
    
    # insert the dbcodes from *code into the original official srcode
    dbsrc = ""
    indent = 4
    onedbprint = False
    
    # make sure the last line which is "" is removed from lst
    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]
    
    # dbprint is enabled to accept both string or int
    srclines = ""
    if type(dbcode) == int:
        srclines = lst[dbcode]
    else:
        srclines = dbcode
    
    # express and insert the dbcode after the srcline under investigation
    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:
            
            # get current l's indentation is enough here, as dbcode is above l
            numindent = len(l) - len(l.strip())
            # attach dbcode above the l under investigation
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes):
                    dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                else:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            # make sure dbprint only written once for multi-srclines under investigation
            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n' # adding this line above dbprint line
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n' # don't forget to add the srcline below dbprint
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'
        
        elif bool(l.strip()) and idx + 1 == len(lst): # handle the last line of srcode
            dbsrc = dbsrc + l
            
        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'
    
    # print out the new srcode
    # for l in dbsrc.split('\n'):
    #     print(l)

    exec(dbsrc, globals().update(env)) # make sure b can access lst from above
    
    # check to see whether the new srcode is created
    # print(f'locals()["src"]: {locals()["src"]}')
    # print(f'locals()["{src.__name__}"]: {locals()[src.__name__]}')
    
    # this is crucial to bring what inside a func namespace into the outside world
    env.update(locals())
    
    return locals()[defaults.orisrc.__name__]

### collect cmt for later printsrcwithidx to print comments together

In [ ]:
def dbprint(src, # the src func name, e.g., foo
            dbcode, # the srclines under investigation, can be either string or int
            cmt:str, # comment
            *codes, # a list of dbcodes
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            env={}): # out environment
    "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

    # make sure the original official src is kept safe and used whenever dbprint is used
    if defaults.orisrc == None:
        defaults.orisrc = src
    else: 
        src = defaults.orisrc
        
    if type(dbcode) == int: defaults.cmts.update({dbcode: cmt})

    printsrc(src, dbcode, cmt, expand)

    dbsrc = ""
    indent = 4
    onedbprint = False

    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]

    srclines = ""
    if type(dbcode) == int:
        srclines = lst[dbcode]
    else:
        srclines = dbcode

    for idx, l in zip(range(len(lst)), lst):
        if bool(l.strip()) and l.strip() in srclines:

            numindent = len(l) - len(l.strip())
            dbcodes = "dbprintinsert("
            count = 1
            for c in codes:
                if count == len(codes):
                    dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                else:
                    dbcodes = dbcodes + '"' + c + '"' + ","
                count = count + 1

            if onedbprint == False:
                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n'
                onedbprint = True
            else:
                dbsrc = dbsrc + l + '\n'

        elif bool(l.strip()) and idx + 1 == len(lst):
            dbsrc = dbsrc + l

        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'

    exec(dbsrc, globals().update(env)) # make sure b can access lst from above

    env.update(locals())

    return locals()[defaults.orisrc.__name__]

### make sure only one line with correct idx is debugged

In [8]:
#|output: asis
#| echo: false
show_doc(dbprint)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L182){target="_blank" style="float:right; font-size:smaller"}

### dbprint

>      dbprint (src, dbcode, cmt:str, *codes, expand:int=2, env={},
>               showdbsrc=False)

Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| src |  |  | the src func name, e.g., foo |
| dbcode |  |  | the srclines under investigation, can be either string or int |
| cmt | str |  | comment |
| codes |  |  |  |
| expand | int | 2 |  |
| env | dict | {} |  |
| showdbsrc | bool | False |  |

### avoid typing "" when there is no codes

In [ ]:
# def dbprint(src, # the src func name, e.g., foo
#             dbcode, # the srclines under investigation, can be either string or int
#             cmt:str, # comment
#             *codes, # a list of dbcodes
#             expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
#             env={}, # out environment
#             showdbsrc=False): # print out dbsrc or not
#     "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

#     # make sure the original official src is kept safe and used whenever dbprint is used
#     if defaults.orisrc == None:
#         defaults.orisrc = src
#     else: 
#         src = defaults.orisrc
        
#     if type(dbcode) == int: defaults.cmts.update({dbcode: cmt})

#     printsrc(src, dbcode, cmt, expand)

#     dbsrc = ""
#     indent = 4
#     onedbprint = False

#     lst = inspect.getsource(src).split('\n')
#     if not bool(lst[-1]): lst = lst[:-1]

#     srclines = ""
#     if type(dbcode) == int:
#         srclines = lst[dbcode]
#     else:
#         srclines = dbcode

#     for idx, l in zip(range(len(lst)), lst):
#         if bool(l.strip()) and l.strip() in srclines:

#             numindent = len(l) - len(l.strip())
#             dbcodes = "dbprintinsert("
#             count = 1
#             for c in codes:
#                 if count == len(codes):
#                     dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
#                 else:
#                     dbcodes = dbcodes + '"' + c + '"' + ","
#                 count = count + 1

#             if onedbprint == False:
#                 dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
#                 dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
#                 dbsrc = dbsrc + l + '\n'
#                 onedbprint = True
#             else:
#                 dbsrc = dbsrc + l + '\n'

#         elif bool(l.strip()) and idx + 1 == len(lst):
#             dbsrc = dbsrc + l

#         elif bool(l.strip()): # make sure pure indentation + \n is ignored
#             dbsrc = dbsrc + l + '\n'

#     if showdbsrc: # added to debug
#         for l in dbsrc.split('\n'):
#             print(l)
    
#     exec(dbsrc, globals().update(env)) # make sure b can access lst from above

#     env.update(locals())

#     return locals()[defaults.orisrc.__name__]

### no more update for dbprint, for the latest see Fastdb.dbprint

### use dbprint to override the original official code without changing its own pyfile

see the example [here](./examples/dbprint.ipynb#make-inspect.signature-to-run-our-dbsrc-code)

```python
sfc = dbprint(_signature_from_callable, "if isinstance(obj, type):", "this is comment", \
              "locals()", "isinstance(obj, type)", "env=g", \
              expand=1, env=g)

# overriding the original official source with our dbsrc, even though rewriting _signature_from_callable inside inspect.py ######################
inspect._signature_from_callable = _signature_from_callable
inspect.signature(Foo) 

```

## dbprintinsert

### Run and display the inserted dbcodes 
for each srcline under investigation, used inside dbprint

In [ ]:
def dbprintinsert(*codes, **env): 
    for c in codes:
    # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
        output = f"{c} => {c} : {eval(c, globals().update(env))}"
        print('{:>157}'.format(output))

In [ ]:
def foo(a):
    a = a*2
    b = a + 1
    c = a * b
    return c

In [ ]:
bool("   ".strip())

False

In [ ]:
g = globals()

In [ ]:
# dbprint(foo, "b = a + 1", "comment", "a", "a + 1", env=g)

AttributeError: 'types.SimpleNamespace' object has no attribute 'orisrc'

In [ ]:
# dbprint(foo, "b = a + 1", "comment", "a", "a + 1", env=g)

In [ ]:
foo(3) #

In [ ]:
# dbprint(foo, "c = a * b", "comment", "b", "b * a", expand=3)

### use locals() inside the dbsrc code to avoid adding env individually

In [ ]:
def dbprintinsert(*codes, env={}): 
    for c in codes:
    # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
        output = f"{c} => {c} : {eval(c, globals().update(env))}"
        print('{:>157}'.format(output))

### enable dbprintinsert to do exec on a block of code

In [9]:
#|output: asis
#| echo: false
show_doc(dbprintinsert)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L254){target="_blank" style="float:right; font-size:smaller"}

### dbprintinsert

>      dbprintinsert (*codes, env={})

In [ ]:
g = globals()

In [ ]:
# foo1 = dbprint(foo, "b = a + 1", "comment", "a", "a + 1")
# foo1(3)

In [ ]:
# foo1 = dbprint(foo, "b = a + 1", "comment", "a", "a + 1", "pprint(a)", "if a > 2:\\n    print('I am here')")
# foo1(3)

In [ ]:
# foo1 = dbprint(foo, "b = a + 1", "comment", "for i in range(3):\\n    print(f'I am {i}')")
# foo1(3)

In [ ]:
# foo1 = dbprint(foo, "b = a + 1", "comment", "for i in range(3): print(f'I am {i}')")
# foo1(3)

In [ ]:
# foo1 = dbprint(foo, "b = a + 1", "comment", "pprint(locals())")
# foo1(3)

In [ ]:
# foo1 = dbprint(foo, "b = a + 1", "comment", "pprint(locals(), width=157)") # width=157 can cause error
# foo1(3)

In [ ]:
from fastcore.meta import delegates

In [ ]:
ls = inspect.getsource(delegates).split('\n')
ls = ls[:-1]
ls

## printrunsrclines() 

It can print out only srclines which actually ran

### Examples

#### simple example

In [ ]:
# def foo(a):
#     if a > 1:
#         a = 1 + 1
#     else:
#         b = a + 1

#### complex example

In [ ]:
def foo(a):
    "this is docs"
    # this is a pure comment
    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        "this is docs"
        b = a + 1
    "this is docs"
    return a

In [ ]:
foo(3)

### insert a line after each srcline to add idx

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""

for idx, l in zip(range(len(srclines)), srclines):
    # if "if" in l or "else" in l or "for" in l:
        
    dbsrc = dbsrc + l + f"\n    srcidx.append({idx})\n" # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

### add correct indentation to each inserted line

#### count the indentation for each srcline

In [ ]:
len("    a = 1") - len("    a = 1".strip())

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"
    dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"   # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

### indentation special case: if, else, for, def

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""
indent = 4

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"

    if "if" in l or "else" in l or "for" in l or "def" in l:
        numindent = numindent + indent
    
    dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

### remove pure comments or docs from dbsrc
Do not insert line for pure comment or pure "\n"

In [ ]:
from pprint import pprint

In [ ]:
for l in srclines:
    pprint(l)

In [ ]:
"# this is a comment".startswith("#")

In [ ]:
"a = 1 # this is comment".startswith("#")

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""
indent = 4

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"

    if "if" in l or "else" in l or "for" in l or "def" in l:
        numindent = numindent + indent
    
    if bool(l): # ignore pure '\n'
        dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'):
    print(l)

In [ ]:
srclines = inspect.getsource(foo).split('\n')
dbsrc = ""
indent = 4

for idx, l in zip(range(len(srclines)), srclines):
    numindent = len(l) - len(l.strip())
    addline = f"srcidx.append({idx})"

    if "if" in l or "else" in l or "for" in l or "def" in l:
        numindent = numindent + indent
    
    if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
        dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

In [ ]:
for l in dbsrc.split('\n'): # now the dbsrc has no pure comment and pure docs
    print(l)

In [ ]:
foo??

In [ ]:
exec(dbsrc) # give life to dbsrc

In [ ]:
foo??

In [ ]:
srcidx = [] #used outside the srcode

In [ ]:
foo(3) # run the example using dbsrc
# foo(-1) # run the example using dbsrc
srcidx # Now it should have all the idx whose srclines have run

### print out the srclines which get run

In [ ]:
for idx, l in zip(range(len(srclines)), srclines):
    if idx in srcidx:
        print(l)

### Make sure all if, else, for get printed

In [ ]:
for idx, l in zip(range(len(srclines)), srclines):
    if idx in srcidx or "for" in l or "if" in l or "else" in l:
        print(l)

### Put all together into the function printrunsrclines()

In [ ]:
def foo(a):
    "this is docs"
    # this is a pure comment
    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        "this is docs"
        b = a + 1
    "this is docs"
    return a

In [ ]:
def printrunsrclines(func):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if" in l or "else" in l or "for" in l or "def" in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line
    
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))
    fool = locals()['foo']
    pprint(fool(3))
    pprint(locals())

     
    # run = "foo(3)"
    exec("fool(3)")
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(foo)

#### no more renaming of foo

In [ ]:
def printrunsrclines(func):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if" in l or "else" in l or "for" in l or "def" in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line
    
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec("foo(3)") # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
def foo(a):

    if a > 1:
        for i in range(3):
            a = i + 1
    else:
        b = a + 1
    return a

In [ ]:
printrunsrclines(foo)

#### add example as a param into the function

In [10]:
#|output: asis
#| echo: false
show_doc(printrunsrclines)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L411){target="_blank" style="float:right; font-size:smaller"}

### printrunsrclines

>      printrunsrclines (func, example)

In [ ]:
printrunsrclines(foo, "foo(-1)")

In [ ]:
printrunsrclines(foo, "foo(2)")

#### improve on search for `if`, else, for, def to avoid errors for more examples

In [ ]:
def printrunsrclines(func, example):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip())
        addline = f"srcidx.append({idx})"

        if "if " in l or "else:" in l or "for " in l or "def " in l:
            numindent = numindent + indent

        if bool(l) and not l.strip().startswith('#') \
        and not (l.strip().startswith('"') and l.strip().endswith('"')): # ignore/remove pure quotations or docs
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

    pprint(dbsrc)
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec(example) # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(alignright, 'alignright("this is me")')

#### remove an empty line with indentation

In [ ]:
lst = """
this is code\n\
     \n\
this is code
""".split('\n')
print(lst)
for l in lst:
    print(bool(l.strip()))

In [ ]:
def printrunsrclines(func, example):
    srclines = inspect.getsource(func).split('\n')
    dbsrc = ""
    indent = 4

    for idx, l in zip(range(len(srclines)), srclines):
        numindent = len(l) - len(l.strip()) # how to strip only the left not the right?????
        addline = f"srcidx.append({idx})"

        if "if " in l or "else:" in l or "for " in l or "def " in l:
            numindent = numindent + indent

        if bool(l.strip()) and not l.strip().startswith('#') \
        and not (l.strip().startswith('"') and l.strip().endswith('"')): 
            dbsrc = dbsrc + l + "\n" + " "*numindent + addline + "\n"  # add srcidx.append(idx) to each line

    pprint(dbsrc)
    srcidx = [] 
    exec(dbsrc, globals().update(locals()))    
    exec(example) # now we can use foo as the new foo 
    print(srcidx)

    for idx, l in zip(range(len(srclines)), srclines):
        if idx in srcidx or "for" in l or "if" in l or "else" in l:
            print(l)

In [ ]:
printrunsrclines(alignright, 'alignright("this is me")')

In [ ]:
pprint(inspect.getsource(printsrc))

In [ ]:
", " in "    env, ".strip()

### make it work

In [ ]:
a=[]
a.append([1,2])
a

In [11]:
#|output: asis
#| echo: false
show_doc(printrunsrclines)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L411){target="_blank" style="float:right; font-size:smaller"}

### printrunsrclines

>      printrunsrclines (src, example, env)

In [ ]:
from fastcore.meta import *
from fastcore.imports import *

In [ ]:
g = globals()

In [ ]:
printrunsrclines(delegates, '', env=g) # make sure to use \\n not \n

In [ ]:
# printsrcwithidx(delegates)

### more difficult examples to test printrunsrc()

## Make fastdebug a class

In [ ]:
class Fastdb():
    
    def __init__(self, 
                 src, # name of src code
                 env): # env = g, as g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env

        
    def dbprint(self, 
                # src, # the src func name, e.g., foo
                dbcode, # the srclines under investigation, can be either string or int
                cmt:str, # comment
                *codes, # a list of dbcodes
                expand:int=2): # span 2 lines of srcode up and down from the srcline investigated
        "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

        src = self.orisrc

        printsrc(src, dbcode, cmt, expand)

        dbsrc = ""
        indent = 4
        onedbprint = False

        lst = inspect.getsource(src).split('\n')
        if not bool(lst[-1]): lst = lst[:-1]

        srclines = ""
        if type(dbcode) == int:
            srclines = lst[dbcode]
        else:
            srclines = dbcode

        for idx, l in zip(range(len(lst)), lst):
            if bool(l.strip()) and l.strip() in srclines:

                numindent = len(l) - len(l.strip())
                dbcodes = "dbprintinsert("
                count = 1
                for c in codes:
                    if count == len(codes) and "env=" in c:
                        dbcodes = dbcodes + c + ")"
                    else:
                        dbcodes = dbcodes + '"' + c + '"' + ","
                    count = count + 1

                if onedbprint == False:
                    dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                    dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                    dbsrc = dbsrc + l + '\n'
                    onedbprint = True
                else:
                    dbsrc = dbsrc + l + '\n'

            elif bool(l.strip()) and idx + 1 == len(lst):
                dbsrc = dbsrc + l

            elif bool(l.strip()): # make sure pure indentation + \n is ignored
                dbsrc = dbsrc + l + '\n'

        exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

        self.outenv.update(locals())

        return locals()[self.orisrc.__name__]
    
    
    def printsrcwithidx(self, 
                        maxlines:int=33, # maximum num of lines per page
                        part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        totallen = 157
        lenidx = 5
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33

        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
        
        for p in range(numparts):
            for idx, l in zip(range(len(lstsrc)), lstsrc):

                if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                    lenl = len(l)
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")

                if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                    print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                    return

In [ ]:
class Fastdb():
    
    def __init__(self, 
                 src, # name of src code
                 env): # env = g, as g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env

        
    def dbprint(self, 
                # src, # the src func name, e.g., foo
                dbcode, # the srclines under investigation, can be either string or int
                cmt:str, # comment
                *codes, # a list of dbcodes
                expand:int=2): # span 2 lines of srcode up and down from the srcline investigated
        "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

        src = self.orisrc

        printsrc(src, dbcode, cmt, expand)

        dbsrc = ""
        indent = 4
        onedbprint = False

        lst = inspect.getsource(src).split('\n')
        if not bool(lst[-1]): lst = lst[:-1]

        srclines = ""
        if type(dbcode) == int:
            srclines = lst[dbcode]
        else:
            srclines = dbcode

        for idx, l in zip(range(len(lst)), lst):
            if bool(l.strip()) and l.strip() in srclines:

                numindent = len(l) - len(l.strip())
                dbcodes = "dbprintinsert("
                count = 1
                for c in codes:
                    if count == len(codes):
                        dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                    else:
                        dbcodes = dbcodes + '"' + c + '"' + ","
                    count = count + 1

                if onedbprint == False:
                    dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                    dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                    dbsrc = dbsrc + l + '\n'
                    onedbprint = True
                else:
                    dbsrc = dbsrc + l + '\n'

            elif bool(l.strip()) and idx + 1 == len(lst):
                dbsrc = dbsrc + l

            elif bool(l.strip()): # make sure pure indentation + \n is ignored
                dbsrc = dbsrc + l + '\n'

        exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

        self.outenv.update(locals())

        return locals()[self.orisrc.__name__]
    
    
    def printsrcwithidx(self, 
                        maxlines:int=33, # maximum num of lines per page
                        part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        totallen = 157
        lenidx = 5
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33

        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
        
        for p in range(numparts):
            for idx, l in zip(range(len(lstsrc)), lstsrc):

                if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                    lenl = len(l)
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")

                if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                    print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                    return

### improve on the line idx readability

In [ ]:
class Fastdb():
    
    def __init__(self, 
                 src, # name of src code
                 env): # env = g, as g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env

        
    def dbprint(self, 
                # src, # the src func name, e.g., foo
                dbcode, # the srclines under investigation, can be either string or int
                cmt:str, # comment
                *codes, # a list of dbcodes
                expand:int=2): # span 2 lines of srcode up and down from the srcline investigated
        "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

        src = self.orisrc

        printsrc(src, dbcode, cmt, expand)

        dbsrc = ""
        indent = 4
        onedbprint = False

        lst = inspect.getsource(src).split('\n')
        if not bool(lst[-1]): lst = lst[:-1]

        srclines = ""
        if type(dbcode) == int:
            srclines = lst[dbcode]
        else:
            srclines = dbcode

        for idx, l in zip(range(len(lst)), lst):
            if bool(l.strip()) and l.strip() in srclines:

                numindent = len(l) - len(l.strip())
                dbcodes = "dbprintinsert("
                count = 1
                for c in codes:
                    if count == len(codes):
                        dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                    else:
                        dbcodes = dbcodes + '"' + c + '"' + ","
                    count = count + 1

                if onedbprint == False:
                    dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                    dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                    dbsrc = dbsrc + l + '\n'
                    onedbprint = True
                else:
                    dbsrc = dbsrc + l + '\n'

            elif bool(l.strip()) and idx + 1 == len(lst):
                dbsrc = dbsrc + l

            elif bool(l.strip()): # make sure pure indentation + \n is ignored
                dbsrc = dbsrc + l + '\n'

        exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

        self.outenv.update(locals())

        return locals()[self.orisrc.__name__]
    
    
    def printsrcwithidx(self, 
                        maxlines:int=33, # maximum num of lines per page
                        part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        totallen = 157
        lenidx = 5
        lspace = 10
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33

        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                elif lenl + lspace >= 100:
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                else:
                    print('{:=<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))
        
        for p in range(numparts):
            for idx, l in zip(range(len(lstsrc)), lstsrc):

                if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                    lenl = len(l)
                    if not bool(l.strip()):
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    elif lenl + lspace >= 100:
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    else:
                        print('{:=<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))

                if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                    print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                    return

### collect cmt from dbprint and print

In [ ]:
class Fastdb():
    
    def __init__(self, 
                 src, # name of src code
                 env): # env = g, as g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env
        self.cmts = {}

        
    def dbprint(self, 
                # src, # the src func name, e.g., foo
                dbcode, # the srclines under investigation, can be either string or int
                cmt:str, # comment
                *codes, # a list of dbcodes
                expand:int=2): # span 2 lines of srcode up and down from the srcline investigated
        "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

        src = self.orisrc
        if type(dbcode) == int: self.cmts.update({dbcode: cmt})

        printsrc(src, dbcode, cmt, expand)

        dbsrc = ""
        indent = 4
        onedbprint = False

        lst = inspect.getsource(src).split('\n')
        if not bool(lst[-1]): lst = lst[:-1]

        srclines = ""
        if type(dbcode) == int:
            srclines = lst[dbcode]
        else:
            srclines = dbcode

        for idx, l in zip(range(len(lst)), lst):
            if bool(l.strip()) and l.strip() in srclines:

                numindent = len(l) - len(l.strip())
                dbcodes = "dbprintinsert("
                count = 1
                for c in codes:
                    if count == len(codes):
                        dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                    else:
                        dbcodes = dbcodes + '"' + c + '"' + ","
                    count = count + 1

                if onedbprint == False:
                    dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                    dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                    dbsrc = dbsrc + l + '\n'
                    onedbprint = True
                else:
                    dbsrc = dbsrc + l + '\n'

            elif bool(l.strip()) and idx + 1 == len(lst):
                dbsrc = dbsrc + l

            elif bool(l.strip()): # make sure pure indentation + \n is ignored
                dbsrc = dbsrc + l + '\n'

                
        exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

        self.outenv.update(locals())

        return locals()[self.orisrc.__name__]
    
    
    def printsrcwithidx(self, 
                        maxlines:int=33, # maximum num of lines per page
                        part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        totallen = 157
        lenidx = 5
        lspace = 10
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
        # cmts = {5:"this is me", 111:"this is me", 14:"this is you this is you this is you this is you this is you this is you this is you this is you "}
        cmts = self.cmts
        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    
                elif lenl + lspace >= 100:
                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                        else:
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    else: 
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        
                else:


                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                      

                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                 
        
        for p in range(numparts):
            for idx, l in zip(range(len(lstsrc)), lstsrc):

                if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                    lenl = len(l)
                    if not bool(l.strip()):
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    elif lenl + lspace >= 100:
                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                            else:
                                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        else: 
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")


                    else:

                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                            else:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                          
                            
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                      

                if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                    print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                    return

### make sure only the line with correct idx is debugged

In [ ]:
class Fastdb():
    
    def __init__(self, 
                 src, # name of src code
                 env): # env = g, as g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env
        self.cmts = {}

        
    def dbprint(self, 
                # src, # the src func name, e.g., foo
                dbcode, # the srclines under investigation, can be either string or int
                cmt:str, # comment
                *codes, # a list of dbcodes
                expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
                showdbsrc=False): # display dbsrc or not
        "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

        src = self.orisrc
        if type(dbcode) == int: self.cmts.update({dbcode: cmt})

        printsrc(src, dbcode, cmt, expand)

        dbsrc = ""
        indent = 4
        onedbprint = False

        lst = inspect.getsource(src).split('\n')
        if not bool(lst[-1]): lst = lst[:-1]

        srclines = ""
        if type(dbcode) == int:
            srclines = lst[dbcode]
        else:
            srclines = dbcode

        for idx, l in zip(range(len(lst)), lst):
            # make sure the line with correct idx is debugged
            if bool(l.strip()) and l.strip() in srclines and idx == dbcode: 

                numindent = len(l) - len(l.strip())
                dbcodes = "dbprintinsert("
                count = 1
                for c in codes:
                    if count == len(codes):
                        dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                    else:
                        dbcodes = dbcodes + '"' + c + '"' + ","
                    count = count + 1

                # if onedbprint == False:
                #     dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                #     dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                #     dbsrc = dbsrc + l + '\n'
                #     onedbprint = True
                # else:
                #     dbsrc = dbsrc + l + '\n'
                
                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n'                

            elif bool(l.strip()) and idx + 1 == len(lst):
                dbsrc = dbsrc + l

            elif bool(l.strip()): # make sure pure indentation + \n is ignored
                dbsrc = dbsrc + l + '\n'
                
        if showdbsrc: # added to debug
            for l in dbsrc.split('\n'):
                print(l)
                
        exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

        self.outenv.update(locals())

        return locals()[self.orisrc.__name__]
    
    
    def printsrcwithidx(self, 
                        maxlines:int=33, # maximum num of lines per page
                        part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        totallen = 157
        lenidx = 5
        lspace = 10
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
        # cmts = {5:"this is me", 111:"this is me", 14:"this is you this is you this is you this is you this is you this is you this is you this is you "}
        cmts = self.cmts
        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    
                elif lenl + lspace >= 100:
                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                        else:
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    else: 
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        
                else:


                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                      

                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                 
        
        for p in range(numparts):
            for idx, l in zip(range(len(lstsrc)), lstsrc):

                if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                    lenl = len(l)
                    if not bool(l.strip()):
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    elif lenl + lspace >= 100:
                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                            else:
                                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        else: 
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")


                    else:

                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                            else:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                          
                            
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                      

                if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                    print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                    return

### having "" or "   " inside codes without causing error

In [ ]:
bool(["   "][0].strip())

In [ ]:
bool("   ".strip())

In [ ]:
lst = ["...", "", "  ", ""]
newlst = []
for i in lst: 
    if bool(i.strip()): newlst.append(i)
newlst

In [ ]:
class Fastdb():
    
    def __init__(self, 
                 src, # name of src code
                 env): # env = g, as g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env
        self.cmts = {}

        
    def dbprint(self, 
                # src, # the src func name, e.g., foo
                dbcode, # the srclines under investigation, can be either string or int
                cmt:str, # comment
                *codes, # a list of dbcodes
                expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
                showdbsrc=False): # display dbsrc or not
        "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

        src = self.orisrc
        if type(dbcode) == int: self.cmts.update({dbcode: cmt})

        printsrc(src, dbcode, cmt, expand)

        dbsrc = ""
        indent = 4
        onedbprint = False

        lst = inspect.getsource(src).split('\n')
        if not bool(lst[-1]): lst = lst[:-1]
        
        newlst = []
        for i in codes: # no matter whether there is "" or "  " in the front or in the middle of codes
            if bool(i.strip()): newlst.append(i)
        codes = newlst

        srclines = ""
        if type(dbcode) == int:
            srclines = lst[dbcode]
        else:
            srclines = dbcode

        for idx, l in zip(range(len(lst)), lst):

            if bool(l.strip()) and l.strip() in srclines and idx == dbcode: 

                if len(codes) > 0: # if the new codes is not empty
                    numindent = len(l) - len(l.strip())
                    dbcodes = "dbprintinsert("
                    count = 1
                    for c in codes:
                        if count == len(codes):
                            dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                        else:
                            dbcodes = dbcodes + '"' + c + '"' + ","
                        count = count + 1

                    dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                    dbsrc = dbsrc + " "*numindent + dbcodes + '\n'

                else:
                    dbsrc = dbsrc + l + '\n'                

            elif bool(l.strip()) and idx + 1 == len(lst):
                dbsrc = dbsrc + l

            elif bool(l.strip()): # make sure pure indentation + \n is ignored
                dbsrc = dbsrc + l + '\n'
                
        if showdbsrc: # added to debug
            for l in dbsrc.split('\n'):
                print(l)
                
        exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

        self.outenv.update(locals())

        return locals()[self.orisrc.__name__]
    
    
    def printsrcwithidx(self, 
                        maxlines:int=33, # maximum num of lines per page
                        part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        totallen = 157
        lenidx = 5
        lspace = 10
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
        # cmts = {5:"this is me", 111:"this is me", 14:"this is you this is you this is you this is you this is you this is you this is you this is you "}
        cmts = self.cmts
        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    
                elif lenl + lspace >= 100:
                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                        else:
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    else: 
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        
                else:


                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                      

                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                 
        
        for p in range(numparts):
            for idx, l in zip(range(len(lstsrc)), lstsrc):

                if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                    lenl = len(l)
                    if not bool(l.strip()):
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    elif lenl + lspace >= 100:
                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                            else:
                                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        else: 
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")


                    else:

                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                            else:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                          
                            
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                      

                if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                    print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                    return

### replace Fastdb.printsrcwithdix with Fastdb.print

In [ ]:
class Fastdb():
    
    def __init__(self, 
                 src, # name of src code
                 env): # env = g, as g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env
        self.cmts = {}

        
    def dbprint(self, 
                # src, # the src func name, e.g., foo
                dbcode, # the srclines under investigation, can be either string or int
                cmt:str, # comment
                *codes, # a list of dbcodes
                expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
                showdbsrc=False): # display dbsrc or not
        "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

        src = self.orisrc
        if type(dbcode) == int: self.cmts.update({dbcode: cmt})

        printsrc(src, dbcode, cmt, expand)

        dbsrc = ""
        indent = 4
        onedbprint = False

        lst = inspect.getsource(src).split('\n')
        if not bool(lst[-1]): lst = lst[:-1]
        
        newlst = []
        for i in codes: # no matter whether there is "" or "  " in the front or in the middle of codes
            if bool(i.strip()): newlst.append(i)
        codes = newlst

        srclines = ""
        if type(dbcode) == int:
            srclines = lst[dbcode]
        else:
            srclines = dbcode

        for idx, l in zip(range(len(lst)), lst):

            if bool(l.strip()) and l.strip() in srclines and idx == dbcode: 

                if len(codes) > 0: # if the new codes is not empty
                    numindent = len(l) - len(l.strip())
                    dbcodes = "dbprintinsert("
                    count = 1
                    for c in codes:
                        if count == len(codes):
                            dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                        else:
                            dbcodes = dbcodes + '"' + c + '"' + ","
                        count = count + 1

                    dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                    dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                    dbsrc = dbsrc + l + '\n'     
                else:
                    dbsrc = dbsrc + l + '\n'                

            elif bool(l.strip()) and idx + 1 == len(lst):
                dbsrc = dbsrc + l

            elif bool(l.strip()): # make sure pure indentation + \n is ignored
                dbsrc = dbsrc + l + '\n'
                
        if showdbsrc: # added to debug
            for l in dbsrc.split('\n'):
                print(l)
                
        exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

        self.outenv.update(locals())

        return locals()[self.orisrc.__name__]
    
    
    def print(self, 
                maxlines:int=33, # maximum num of lines per page
                part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        totallen = 157
        lenidx = 5
        lspace = 10
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
        # cmts = {5:"this is me", 111:"this is me", 14:"this is you this is you this is you this is you this is you this is you this is you this is you "}
        cmts = self.cmts
        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    
                elif lenl + lspace >= 100:
                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                        else:
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    else: 
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        
                else:


                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                      

                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                 
        
        for p in range(numparts):
            for idx, l in zip(range(len(lstsrc)), lstsrc):

                if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                    lenl = len(l)
                    if not bool(l.strip()):
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    elif lenl + lspace >= 100:
                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                            else:
                                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        else: 
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")


                    else:

                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                            else:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                          
                            
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                      

                if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                    print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                    return

### add idx to dbsrc when showdbsrc=True

In [ ]:
class Fastdb():
    
    def __init__(self, 
                 src, # name of src code
                 env): # env = g, as g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env
        self.cmts = {}

        
    def dbprint(self, 
                # src, # the src func name, e.g., foo
                dbcode, # the srclines under investigation, can be either string or int
                cmt:str, # comment
                *codes, # a list of dbcodes
                expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
                showdbsrc=False): # display dbsrc or not
        "Insert dbcodes under srclines under investigation, and create a new dbsrc function to replace the official one"

        src = self.orisrc
        if type(dbcode) == int: self.cmts.update({dbcode: cmt})

        printsrc(src, dbcode, cmt, expand)

        dbsrc = ""
        indent = 4
        onedbprint = False

        lst = inspect.getsource(src).split('\n')
        if not bool(lst[-1]): lst = lst[:-1]
        
        newlst = []
        for i in codes: # no matter whether there is "" or "  " in the front or in the middle of codes
            if bool(i.strip()): newlst.append(i)
        codes = newlst

        srclines = ""
        if type(dbcode) == int:
            srclines = lst[dbcode]
        else:
            srclines = dbcode

        for idx, l in zip(range(len(lst)), lst):

            if bool(l.strip()) and l.strip() in srclines and idx == dbcode: 

                if len(codes) > 0: # if the new codes is not empty
                    numindent = len(l) - len(l.strip())
                    dbcodes = "dbprintinsert("
                    count = 1
                    for c in codes:
                        if count == len(codes):
                            dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                        else:
                            dbcodes = dbcodes + '"' + c + '"' + ","
                        count = count + 1

                    dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                    dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                    dbsrc = dbsrc + l + '\n'     
                else:
                    dbsrc = dbsrc + l + '\n'                

            elif bool(l.strip()) and idx + 1 == len(lst):
                dbsrc = dbsrc + l

            elif bool(l.strip()): # make sure pure indentation + \n is ignored
                dbsrc = dbsrc + l + '\n'
                
        if showdbsrc: # added to debug
            totallen = 157
            lenidx = 5
            dblst = dbsrc.split('\n')
            for idx, l in zip(range(len(dblst)), dblst):
                lenl = len(l)
                if "dbprintinsert" in l: 
                    print(l + "="*(totallen-lenl-lenidx) + "(db)")
                else:
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                
        exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

        self.outenv.update(locals())

        return locals()[self.orisrc.__name__]
    
    
    def print(self, 
                maxlines:int=33, # maximum num of lines per page
                part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        totallen = 157
        lenidx = 5
        lspace = 10
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
        # cmts = {5:"this is me", 111:"this is me", 14:"this is you this is you this is you this is you this is you this is you this is you this is you "}
        cmts = self.cmts
        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    
                elif lenl + lspace >= 100:
                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                        else:
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    else: 
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        
                else:


                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                      

                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                 
        
        for p in range(numparts):
            for idx, l in zip(range(len(lstsrc)), lstsrc):

                if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                    lenl = len(l)
                    if not bool(l.strip()):
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    elif lenl + lspace >= 100:
                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                            else:
                                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        else: 
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")


                    else:

                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                            else:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                          
                            
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                      

                if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                    print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                    return

### not load the inner locals() to outenv can prevent mysterious printing of previous db messages

In [ ]:
class Fastdb():
    "Create a Fastdebug class which has two functionalities: dbprint and print."
    def __init__(self, 
                 src, # name of src code
                 env): # env = g, as g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env
        self.cmts = {}

        
    def dbprint(self, 
                # src, # the src func name, e.g., foo
                dbcode:int, # a srcline under investigation, can be either string or int
                cmt:str, # comment
                *codes, # a list of expressions (str) you write to be evaluated above the srcline
                expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
                showdbsrc:bool=False): # display dbsrc
        "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"

        src = self.orisrc
        if type(dbcode) == int: self.cmts.update({dbcode: cmt})

        printsrc(src, dbcode, cmt, expand)

        dbsrc = ""
        indent = 4
        onedbprint = False

        lst = inspect.getsource(src).split('\n')
        if not bool(lst[-1]): lst = lst[:-1]
        
        newlst = []
        for i in codes: # no matter whether there is "" or "  " in the front or in the middle of codes
            if bool(i.strip()): newlst.append(i)
        codes = newlst

        srclines = ""
        if type(dbcode) == int:
            srclines = lst[dbcode]
        else:
            srclines = dbcode

        for idx, l in zip(range(len(lst)), lst):

            if bool(l.strip()) and l.strip() in srclines and idx == dbcode: 

                if len(codes) > 0: # if the new codes is not empty
                    numindent = len(l) - len(l.strip())
                    dbcodes = "dbprintinsert("
                    count = 1
                    for c in codes:
                        if count == len(codes):
                            dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                        else:
                            dbcodes = dbcodes + '"' + c + '"' + ","
                        count = count + 1

                    dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                    dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                    dbsrc = dbsrc + l + '\n'     
                else:
                    dbsrc = dbsrc + l + '\n'                

            elif bool(l.strip()) and idx + 1 == len(lst):
                dbsrc = dbsrc + l

            elif bool(l.strip()): # make sure pure indentation + \n is ignored
                dbsrc = dbsrc + l + '\n'
                
        if showdbsrc: # added to debug
            totallen = 157
            lenidx = 5
            dblst = dbsrc.split('\n')
            for idx, l in zip(range(len(dblst)), dblst):
                lenl = len(l)
                if "dbprintinsert" in l: 
                    print(l + "="*(totallen-lenl-lenidx) + "(db)")
                else:
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                
        exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

        # self.outenv.update(locals())
        # self.outenv.update({self.orisrc.__name__: locals()[self.orisrc.__name__]})

        return locals()[self.orisrc.__name__]
    
    
    def print(self, 
                maxlines:int=33, # maximum num of lines per page
                part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
        "Print the source code in whole or parts with idx and comments you added with dbprint along the way."
        
        totallen = 157
        lenidx = 5
        lspace = 10
        lstsrc = inspect.getsource(self.orisrc).split('\n')
        numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
        # cmts = {5:"this is me", 111:"this is me", 14:"this is you this is you this is you this is you this is you this is you this is you this is you "}
        cmts = self.cmts
        if part == 0: 
            for idx, l in zip(range(len(lstsrc)), lstsrc):
                lenl = len(l)
                
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    
                elif lenl + lspace >= 100:
                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                        else:
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    else: 
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        
                else:


                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                      

                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                 
        
        for p in range(numparts):
            for idx, l in zip(range(len(lstsrc)), lstsrc):

                if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                    lenl = len(l)
                    if not bool(l.strip()):
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    elif lenl + lspace >= 100:
                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                            else:
                                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                        else: 
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")


                    else:

                        if bool(cmts):
                            cmtidx = [cmt[0] for cmt in list(cmts.items())]
                            if idx in cmtidx:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                            else:
                                print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                          
                            
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                      

                if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                    print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                    return

### using @patch to enable docs for instance methods like [`dbprint`](https://EmbraceLife.github.io/fastdebug/core.html#dbprint) and `print`

In [ ]:
class Fastdb():
    "Create a Fastdebug class which has two functionalities: dbprint and print."
    def __init__(self, 
                 src, # name of src code you are exploring
                 env): # env variables needed for exploring the source code, e.g., g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env
        self.cmts = {}

In [12]:
#|output: asis
#| echo: false
show_doc(Fastdb)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L482){target="_blank" style="float:right; font-size:smaller"}

### Fastdb

>      Fastdb (src, env)

Create a Fastdebug class which has two functionalities: dbprint and print.

|    | **Details** |
| -- | ----------- |
| src | name of src code you are exploring |
| env | env variables needed for exploring the source code, e.g., g = globals() |

In [ ]:
@patch
def dbprint(self:Fastdb, 
            dbcode:int, # a srcline under investigation, can be either string or int
            cmt:str, # comment added to the srcline
            *codes, # a list of expressions (str) you write to be evaluated above the srcline
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            showdbsrc:bool=False): # display dbsrc
    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"

    src = self.orisrc
    if type(dbcode) == int: self.cmts.update({dbcode: cmt})

    printsrc(src, dbcode, cmt, expand)

    dbsrc = ""
    indent = 4
    onedbprint = False

    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]

    newlst = []
    for i in codes: # no matter whether there is "" or "  " in the front or in the middle of codes
        if bool(i.strip()): newlst.append(i)
    codes = newlst

    srclines = ""
    if type(dbcode) == int:
        srclines = lst[dbcode]
    else:
        srclines = dbcode

    for idx, l in zip(range(len(lst)), lst):

        if bool(l.strip()) and l.strip() in srclines and idx == dbcode: 

            if len(codes) > 0: # if the new codes is not empty
                numindent = len(l) - len(l.strip())
                dbcodes = "dbprintinsert("
                count = 1
                for c in codes:
                    if count == len(codes):
                        dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                    else:
                        dbcodes = dbcodes + '"' + c + '"' + ","
                    count = count + 1

                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n'     
            else:
                dbsrc = dbsrc + l + '\n'                

        elif bool(l.strip()) and idx + 1 == len(lst):
            dbsrc = dbsrc + l

        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'

    if showdbsrc: # added to debug
        totallen = 157
        lenidx = 5
        dblst = dbsrc.split('\n')
        for idx, l in zip(range(len(dblst)), dblst):
            lenl = len(l)
            if "dbprintinsert" in l: 
                print(l + "="*(totallen-lenl-lenidx) + "(db)")
            else:
                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")

    exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above

    return locals()[self.orisrc.__name__]

In [13]:
#|output: asis
#| echo: false
show_doc(Fastdb.dbprint)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L494){target="_blank" style="float:right; font-size:smaller"}

### Fastdb.dbprint

>      Fastdb.dbprint (dbcode:int, cmt:str, *codes, expand:int=2,
>                      showdbsrc:bool=False)

Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| dbcode | int |  | a srcline under investigation, can be either string or int |
| cmt | str |  | comment added to the srcline |
| codes |  |  |  |
| expand | int | 2 |  |
| showdbsrc | bool | False |  |

In [ ]:
@patch
def print(self:Fastdb, 
            maxlines:int=33, # maximum num of lines per page
            part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
    "Print the source code in whole or parts with idx and comments you added with dbprint along the way."

    totallen = 157
    lenidx = 5
    lspace = 10
    lstsrc = inspect.getsource(self.orisrc).split('\n')
    numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
    cmts = self.cmts
    if part == 0: 
        for idx, l in zip(range(len(lstsrc)), lstsrc):
            lenl = len(l)

            if not bool(l.strip()):
                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")

            elif lenl + lspace >= 100:
                if bool(cmts):
                    cmtidx = [cmt[0] for cmt in list(cmts.items())]
                    if idx in cmtidx:
                        print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                    else:
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                else: 
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")

            else:


                if bool(cmts):
                    cmtidx = [cmt[0] for cmt in list(cmts.items())]
                    if idx in cmtidx:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                      

                else:
                    print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                 

    for p in range(numparts):
        for idx, l in zip(range(len(lstsrc)), lstsrc):

            if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                lenl = len(l)
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                elif lenl + lspace >= 100:
                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                        else:
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    else: 
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")


                else:

                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                          

                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                      

            if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                return

In [14]:
#|output: asis
#| echo: false
show_doc(Fastdb.print)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core.py#L570){target="_blank" style="float:right; font-size:smaller"}

### Fastdb.print

>      Fastdb.print (maxlines:int=33, part:int=0)

Print the source code in whole or parts with idx and comments you added with dbprint along the way.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| maxlines | int | 33 | maximum num of lines per page |
| part | int | 0 | if the src is more than 33 lines, then divide the src by 33 into a few parts |

### use dbprint to override the original official code without changing its own pyfile

see the example [here](./examples/dbprint.ipynb#make-inspect.signature-to-run-our-dbsrc-code)

```python
dbsig = sig.dbprint(29, "why has to unwrap?", "hasattr(obj, '__signature__')")
inspect._signature_from_callable = dbsig
pprint(inspect.signature(Foo))
sig.print(part=1)
```

### use guide on Fastdb.dbprint

1. don't use for the line start with `elif`, as putting [`dbprintinsert`](https://EmbraceLife.github.io/fastdebug/core.html#dbprintinsert) above `elif` without indentation will cause syntax error. I am not sure whether I need to fix it now.

see example [here](./examples/Fastdb.ipynb)

test it with example [here](./examples/print.ipynb)